In [ ]:
#import json
#import random
#import re
#import numpy as np
#import tensorflowjs as tfjs
#from tensorflow.keras.utils import to_categorical
#from tensorflow.keras import regularizers

In [34]:
from model.Corpus import Corpus
from model.Model import Model
from model.Embeddings import Embeddings

# Dimensions in the GloVe embedding
embedding_dim = 100
# Enforced max and min length of an example
max_length = 60

trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

num_epochs = 4
training_size= 69000
test_portion=.1

corpus = Corpus(
    "training_data/augmented_training_labels.csv",
    training_size,
    max_length,
    padding_type,
    trunc_type,
    test_portion
).initialize()

embeddings = Embeddings(
    "glove.6B.100d.txt",
    embedding_dim,
    corpus
).activate()

In [35]:
classifier = Model(
    corpus,
    embeddings,
    max_length,
    num_epochs,
    5
).run()


Epoch 1/4
1941/1941 [==============================] - 49s 25ms/step - loss: 0.0622 - accuracy: 0.9795 - val_loss: 0.0262 - val_accuracy: 0.9929
Epoch 2/4
1941/1941 [==============================] - 51s 26ms/step - loss: 0.0254 - accuracy: 0.9918 - val_loss: 0.0230 - val_accuracy: 0.9929
Epoch 3/4
1941/1941 [==============================] - 52s 27ms/step - loss: 0.0202 - accuracy: 0.9931 - val_loss: 0.0232 - val_accuracy: 0.9926
Epoch 4/4
1941/1941 [==============================] - 52s 27ms/step - loss: 0.0160 - accuracy: 0.9942 - val_loss: 0.0213 - val_accuracy: 0.9939


In [40]:
from pprint import pprint
import csv
from scrapers.ScraperResult import ScraperResult
from page_processors import *

total_payload = []
new_links = [
    "https://www.acouplecooks.com/gin-fizz-cocktail/"
]

with open('newlabels.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    total = {}
    counter = 0
    for link in new_links:
        print(link)
        #try:
        data = process_page(link,writer,classifier,corpus,max_length,padding_type,trunc_type)
        this_recipe = ScraperResult(
            link.split("/")[-1],
            list(map(lambda x: x[0], lasso_inward(data,4,6,1,7)[0])),
            list(map(lambda x: x[0], lasso_inward(data,4,6,2,7)[0])),
            list(map(lambda x: x[0], lasso_inward(data,4,6,3,7)[0])),
            list(map(lambda x: x[0], lasso_inward(data,4,6,4,7)[0]))
        )
        total_payload.append(this_recipe.get_payload())
        payload = {
            "title": link,
            "steps": list(map(lambda x: x[0], lasso_inward(data,4,6,1,7)[0])),
            "ingredients": list(map(lambda x: x[0], lasso_inward(data,4,6,2,7)[0])),
            "yield": list(map(lambda x: x[0], lasso_inward(data,4,6,3,7)[0])),
            "timing": list(map(lambda x: x[0], lasso_inward(data,4,6,4,7)[0]))
        }
        total[str(counter)] = payload
        counter += 1
        #except:
        #    print("link " + link + " could not be opened")




https://www.acouplecooks.com/gin-fizz-cocktail/


In [37]:
clean_payload = list(filter(lambda x: x["ingredients"] != [''] and x["steps"] != [''],total_payload))
print(len(clean_payload))
print(len(total_payload))
import json
with open('bbc_cocktails_json.txt', 'w') as output_file:
    json.dump(clean_payload, output_file)
    

235
236


In [41]:
from scrapers.CocktailLinkScraper import CocktailLinkScraper
cs = CocktailLinkScraper()


[{'ingredients': ['2 ounces  gin',
                  '3/4 ounce  lemon juice',
                  '1/2 ounce  simple syrup or maple syrup',
                  '1 egg white*',
                  'Garnish with lemon twist',
                  'Soda water'],
  'steps': ['How to make a Gin Fizz? You’ll shake together the gin, fresh '
            'lemon juice, syrup, and egg white in a cocktail shaker without '
            'ice first, then add ice and pour it into two glasses.',
            'The trick to the foam topping: a dry shake!',
            'Shake the drink ingredients without ice first. This lets the '
            'protein in the egg begin to form foam, instead of being diluted '
            'by the ice.',
            'Then add ice and shake again. This cools the drink and '
            'strengthens the foam. Strain it into the glass and you’ll get a '
            'thick, white frothy layer.',
            'Since 2 tablespoons of aquafaba equal 1 egg white, you can use 2 '
            '